In [4]:
import pandas as pd

df = pd.read_parquet('green_tripdata_2025-11.parquet')

df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [5]:
# Filter for November 2025 only and count trips with trip_distance <= 1 mile
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
november_df = df[(df['lpep_dropoff_datetime'].dt.year == 2025) & (df['lpep_dropoff_datetime'].dt.month == 11)]
count = (november_df['trip_distance'] <= 1).sum()
count

8007

In [6]:
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
filtered_df = df[df['trip_distance'] < 100]

# Find the trip with the longest distance
max_distance_row = filtered_df.loc[filtered_df['trip_distance'].idxmax()]

# Extract the pickup date
pickup_day = max_distance_row['lpep_pickup_datetime'].date()

print(f"Pickup day with longest trip distance: {pickup_day}")

Pickup day with longest trip distance: 2025-11-14


In [11]:
# Filter for November 18th, 2025
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
nov_18_df = df[df['lpep_pickup_datetime'].dt.date == pd.to_datetime('2025-11-18').date()]

# Group by pickup zone (PULocationID) and sum total_amount
zone_totals = nov_18_df.groupby('PULocationID')['total_amount'].sum().sort_values(ascending=False)

# Join pickup zone from csv
df_zones = pd.read_csv('taxi_zone_lookup.csv')
df_zones = df_zones[['LocationID', 'Zone']].set_index('LocationID')
zone_totals = zone_totals.to_frame().join(df_zones, how='left')

# Display the top pickup zone
print("Top pickup zone by total_amount on November 18th, 2025:")
print(zone_totals.head(1))

Top pickup zone by total_amount on November 18th, 2025:
              total_amount               Zone
PULocationID                                 
74                 9281.92  East Harlem North


In [12]:
# For passengers picked up in "East Harlem North" in November 2025, find drop off zone with largest tip
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])

# Load zone lookup
df_zones = pd.read_csv('taxi_zone_lookup.csv')

# Find LocationID for "East Harlem North"
east_harlem_north_id = df_zones[df_zones['Zone'] == 'East Harlem North']['LocationID'].values[0]

# Filter for pickups in East Harlem North during November 2025
east_harlem_df = df[
    (df['PULocationID'] == east_harlem_north_id) & 
    (df['lpep_pickup_datetime'].dt.year == 2025) & 
    (df['lpep_pickup_datetime'].dt.month == 11)
]

# Find the trip with the largest tip
if len(east_harlem_df) > 0:
    max_tip_idx = east_harlem_df['tip_amount'].idxmax()
    max_tip_row = east_harlem_df.loc[max_tip_idx]
    
    # Get drop off zone name
    dropoff_zone_id = max_tip_row['DOLocationID']
    dropoff_zone_name = df_zones[df_zones['LocationID'] == dropoff_zone_id]['Zone'].values[0]
    
    print(f"Drop off zone with largest tip: {dropoff_zone_name}")
    print(f"Tip amount: ${max_tip_row['tip_amount']:.2f}")
else:
    print("No trips found for East Harlem North in November 2025")

Drop off zone with largest tip: Yorkville West
Tip amount: $81.89
